## LLM

In [1]:
import json

# Load configuration from JSON file
with open('../config.json') as config_file:
    config = json.load(config_file)

In [2]:
db_scheme = config.get("DB_SCHEME")
palm_api_key = config.get("API_KEY")
user = config.get("user")
password = config.get("password")
dbname = config.get("dbname")

In [3]:
from llama_index.llms.palm import PaLM

model = PaLM(api_key=palm_api_key)

c:\Users\TurnerZ\Documents\GitHub\DBSage_Navigator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Data

In [4]:
from llama_index import download_loader

DatabaseReader = download_loader('DatabaseReader')

In [ ]:
reader = DatabaseReader(
    scheme = "postgresql", # Database Scheme
    host = "localhost", # Database Host
    port = "5432", # Database Port
    user = user, # Database User
    password = password, # Database Password
    dbname = dbname, # Database Name
)

query = """
SELECT * FROM public.employee_data
"""

documents = reader.load_data(query=query)